# Classifying Images with CNNs

In this exercise you will design a Convolutional Neural Network (CNN) for MNIST. CNNs are the workhorses of modern computer vision.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils

## CNN

Let's build a convolutional model! For this, we need to have the data in its original shape. Also note that when we reshape the data below, we add a dimension of 1 - this is the number of **channels** in the image, which is just 1 because these are grayscale images. If they were color, this would be 3 for RGB. Make sure you understand how and why we are preparing the data below.

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 28 x 28 image
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print X_train.shape

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

(60000, 28, 28, 1)


### Exercise 1 - design a CNN for MNIST

It is up to you what the model will be. Here are some things you need to decide:
* how many convolutional layers?
* what spatial size will your convolutions be?
* how many channels will your convolutions be?
* what nonlinearity will you use?
* will you use pooling? what type?
* how many fully-connected layers will you have?
* will you use dropout?
* what batch size?

Keras provides a special layer called `Flatten` to flatten the convolutional features into a vector before the fully-connected layers. You should look at the documentation for Keras's convolutional layers: http://keras.io/layers/convolutional/. In particular, you may want to look at `Convolution2D`, `MaxPooling2D`, `AveragePooling2D`, `Flatten`, and `Dropout`. For this problem, you make want to use the `'rmsprop'` optimizer - it is an algorithm that adapts the learning rate during learning for you automatically.

Can you get to 98% accuracy? You shouldn't need more than a few epochs to do pretty well.

Suggestions:
* Try using at least 2 convolutional layers. This should get you off to a good start and it will come in handy later.

In [5]:
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, Flatten

print 'input shape', X_train.shape

model = Sequential()

# 3x3 convolution on top, with 10  filtera:
model.add(Convolution2D(10, 3, 3, border_mode='same', input_shape=(28, 28, 1)))  # tf ordering
print '1st CNN layer', model.output_shape 

model.add(MaxPooling2D())

# 3x3 convolution on top, with 32 output filters:
model.add(Convolution2D(32, 3, 3, border_mode='same'))
print '2nd CNN layer', model.output_shape 

model.add(MaxPooling2D())

model.add(Flatten())
print 'Flattened CNN layer', model.output_shape 

model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Fit your model
hist = model.fit(X_train, y_train, validation_split=0.2,  nb_epoch=10, batch_size=32, verbose=2)

input shape (60000, 28, 28, 1)
1st CNN layer (None, 28, 28, 10)
2nd CNN layer (None, 14, 14, 32)
Flattened CNN layer (None, 1568)
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
13s - loss: 0.1879 - acc: 0.9431 - val_loss: 0.0649 - val_acc: 0.9804
Epoch 2/10
13s - loss: 0.0661 - acc: 0.9795 - val_loss: 0.0550 - val_acc: 0.9842
Epoch 3/10
15s - loss: 0.0508 - acc: 0.9851 - val_loss: 0.0542 - val_acc: 0.9864
Epoch 4/10
16s - loss: 0.0406 - acc: 0.9871 - val_loss: 0.0531 - val_acc: 0.9860
Epoch 5/10
16s - loss: 0.0331 - acc: 0.9900 - val_loss: 0.0572 - val_acc: 0.9853
Epoch 6/10
15s - loss: 0.0285 - acc: 0.9907 - val_loss: 0.0504 - val_acc: 0.9878
Epoch 7/10
15s - loss: 0.0239 - acc: 0.9922 - val_loss: 0.0570 - val_acc: 0.9877
Epoch 8/10
15s - loss: 0.0232 - acc: 0.9930 - val_loss: 0.0557 - val_acc: 0.9887
Epoch 9/10
15s - loss: 0.0198 - acc: 0.9941 - val_loss: 0.0593 - val_acc: 0.9883
Epoch 10/10
15s - loss: 0.0178 - acc: 0.9943 - val_loss: 0.0653 - val_acc: 0.9891


In [6]:
score = model.evaluate(X_test, y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.051031891813250602)
('Test accuracy:', 0.99019999999999997)


## Saving a trained model

Keras allows you to save and restore models. The model structure can be saved as JSON and the parameters as HDF5. Save your trained model for later:

In [7]:
with open('mnist_cnn.json', 'w') as f:
    f.write(model.to_json())

model.save_weights('mnist_cnn_weights.h5')